In [ ]:
import os
import numpy as np
import torch

In [ ]:
def random_sample(data, sample_size):
    
    # Check if the sample size is greater than the number of points in the data
    if sample_size > data.shape[0]:
        raise ValueError("Sample size exceeds the number of points in the data")

    random_indices = np.random.choice(data.shape[0], sample_size, replace=False)
    sampled_data = data[random_indices]

    return sampled_data

In [ ]:
class Loader:
    def __init__(self, pathpattern1="/bigdata/hplsim/aipp/Jeyhun/khi/particle_box/40_80_80_160_0_2/{}.npy",
                 pathpattern2="/bigdata/hplsim/aipp/Jeyhun/khi/part_rad/radiation_ex/{}.npy",
                 t0=0,
                 t1=100,
                 timebatchsize=20,
                 particlebatchsize=10240):
        
        self.pathpattern1 = pathpattern1
        self.pathpattern2 = pathpattern2
                
        self.t0 = t0
        self.t1 = t1
        
        self.timebatchsize = timebatchsize
        self.particlebatchsize = particlebatchsize

        num_files = t1 - t0
        missing_files = [i for i in range(t0, t1) if not os.path.exists(pathpattern1.format(i))]
        num_missing = len(missing_files)
        all_files_exist = num_missing == 0

        if all_files_exist:
            print("All {} files from {} to {} exist in the directory.".format(num_files, t0, t1))
        else:
            print("{} files are missing out of {} in the directory.".format(num_missing, num_files))

    def __len__(self):
        return self.t1 - self.t0
        
    def __getitem__(self, idx):
        
        class Epoch:
            def __init__(self, loader, t0, t1, timebatchsize=20, particlebatchsize=10240):
                self.perm = torch.randperm(len(loader))
                self.loader = loader
                self.t0 = t0
                self.t1 = t1
                self.timebatchsize = timebatchsize
                self.particlebatchsize = particlebatchsize

            def __len__(self):
                return len(self.loader) // self.timebatchsize
        
            def __getitem__(self, timebatch):
                i = self.timebatchsize*timebatch
                bi = self.perm[i:i+self.timebatchsize]
                radiation = []
                particles = []
                for time in bi:
                    index = time + self.t0
                    
                    # Load particle data
                    p = np.load(self.loader.pathpattern1.format(index), allow_pickle = True)
                    # Load radiation data
                    r = torch.from_numpy(np.load(self.loader.pathpattern2.format(index)).astype(np.cfloat))
                    
                    # Exclude the blacklisted box if specified
                    # if self.blacklist_box is not None:
                    #     p = np.delete(p, self.blacklist_box, axis=0)
                    #     r = np.delete(r, self.blacklist_box, axis=0)
                    
                    #p = [self.loader.normalisation.normalize_data(element, method=self.loader.norm_method) for element in p]
                    p = np.array(p, dtype=object)
                    
                    # random sample N points from each box
                    p = [random_sample(element, sample_size=150000) for element in p]
                    p = torch.from_numpy(np.array(p, dtype = np.float32))
                    
                    # ampliudes in each direction
                    amp_x = torch.abs(r[:, 0, :]).to(torch.float32)
                    amp_y = torch.abs(r[:, 1, :]).to(torch.float32)
                    amp_z = torch.abs(r[:, 2, :]).to(torch.float32)
                    
                    #spectra
                    r = amp_x**2 + amp_y**2 + amp_z**2
                    
                    #log transformation
                    #r = torch.log(r+1)

                    particles.append( p )
                    radiation.append(r)
                
                particles = torch.cat(particles)
                radiation = torch.cat(radiation)
                
                class Timebatch:
                    def __init__(self, particles, radiation, batchsize):
                        self.batchsize = batchsize
                        self.particles = particles
                        self.radiation = radiation
                        
                        self.perm = torch.randperm(self.radiation.shape[0])
                        
                    def __len__(self):
                        return self.radiation.shape[0] // self.batchsize

                    def __getitem__(self, batch):
                        i = self.batchsize*batch
                        bi = self.perm[i:i+self.batchsize]
                    
                        return self.particles[bi], self.radiation[bi]
                
                return Timebatch(particles, radiation, self.particlebatchsize)
                    
        return Epoch(self, self.t0, self.t1, self.timebatchsize, self.particlebatchsize)


In [ ]:
pathpattern1 = "/bigdata/hplsim/aipp/Jeyhun/khi/part_rad/particle_002/{}.npy"
pathpattern2 = "/bigdata/hplsim/aipp/Jeyhun/khi/part_rad/radiation_ex_002/{}.npy"
    
l = Loader(pathpattern1=pathpattern1,
           pathpattern2=pathpattern2,
           t0=1000,
           t1=2001,
           timebatchsize=4,
           particlebatchsize=32)

In [ ]:
epoch = l[0]
for i_epoch in range(0, 100):   
    loss_overall = []
    for tb in range(len(epoch)):
        loss_avg = []
        timebatch = epoch[tb]
        
        for b in range(len(timebatch)):

            phase_space, radiation = timebatch[b]
            
            print('phase_space', phase_space.shape)
            print('radiation', radiation.shape)